In [234]:
import os
os.chdir('/home/nmduy/CA683/Credit Project')
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_auc_score 
from sklearn.metrics import auc
from sklearn import preprocessing
import pandas as pd
import numpy as np
import random
import json
from sklearn import metrics
random.seed(1509)

## Read data
Read filtered csv file (remove features containing too much missing values) and encode the categorical variables.

In [249]:
##### READ DATA #####
data = pd.read_csv('data/filtered_application_train.csv')
data_remove_id = data.drop(['SK_ID_CURR'], axis=1) if 'SK_ID_CURR' in list(data.columns) else data.copy()
data_remove_id = data_remove_id.drop(['DAYS_EMPLOYED_ANOM'], axis=1) if 'DAYS_EMPLOYED_ANOM' in list(data_remove_id.columns) else data_remove_id.copy()
data_remove_id = data_remove_id.drop(['Unnamed: 0'], axis=1) if 'Unnamed: 0' in list(data_remove_id.columns) else data_remove_id.copy()


In [236]:
ft_types = dict(data_remove_id.dtypes)

categorical_ft = []
numerical_ft = []
for ft, val in ft_types.items():
    if val == np.object:
        categorical_ft += [ft]
    else:
        numerical_ft += [ft]


select_ft = ['TARGET', 'EXT_SOURCE_3', 'EXT_SOURCE_2', 'DAYS_EMPLOYED', 'DAYS_ID_PUBLISH', 'FLOORSMAX_AVG', 
             'DAYS_REGISTRATION', 'AMT_GOODS_PRICE', 'DEF_30_CNT_SOCIAL_CIRCLE', 'HOUR_APPR_PROCESS_START', 
             'AMT_REQ_CREDIT_BUREAU_YEAR', 'CNT_CHILDREN', 'AMT_REQ_CREDIT_BUREAU_MON']

drop_numerical_ft = [x for x in numerical_ft if x not in select_ft]
remain_numerical_ft = [x for x in numerical_ft if x not in drop_numerical_ft]
print((len(drop_numerical_ft), len(remain_numerical_ft)))
data_remove_id = data_remove_id.drop(drop_numerical_ft, axis=1)

(53, 12)


In [250]:
encoded_data = pd.get_dummies(data_remove_id)

print(encoded_data.shape)

##### DROP NA #####
encoded_data = encoded_data.dropna()
encoded_data = encoded_data.reset_index(drop=True)
print(encoded_data.shape)

(307511, 180)
(245140, 180)


In [24]:
np.sum(np.isfinite(encoded_data).sum()-190079)

-3

## MAKE THE DATASET
Create subset and maybe make it balance between classes within classes

In [251]:
##### MAKE IT BALANCE #####
random.seed(1509)

numb_pos_samples = encoded_data['TARGET'].sum()
numb_neg_samples = len(encoded_data) - numb_pos_samples
portion_pos_train = 0.8
portion_pos_val = 0.1
numb_pos_train = int(numb_pos_samples * portion_pos_train)
numb_pos_val = int(numb_pos_samples * portion_pos_val)
numb_pos_test = numb_pos_samples - numb_pos_train - numb_pos_val

index_pos = encoded_data.index[encoded_data['TARGET'] == 1].tolist()
index_neg = [i for i in range(len(encoded_data)) if i not in index_pos]

data_pos = encoded_data.iloc[np.asarray(index_pos), :]
data_neg = encoded_data.iloc[np.asarray(index_neg), :]

##### shuffle #####
data_pos = data_pos.sample(frac=1)
data_neg = data_neg.sample(frac=1)

data_pos_train = data_pos.iloc[0:numb_pos_train,:]
data_pos_val = data_pos.iloc[numb_pos_train:numb_pos_train + numb_pos_val, :]
data_pos_test = data_pos.iloc[numb_pos_train + numb_pos_val:, :]

data_neg_train = data_neg.iloc[0:int(numb_pos_train*11), :]
data_neg_val = data_neg.iloc[int(numb_pos_train*11):-numb_pos_test, :]
data_neg_test = data_neg.iloc[-numb_pos_test:, :]

data_train = pd.concat([data_pos_train, data_neg_train], ignore_index=True)
data_val = pd.concat([data_pos_val, data_neg_val], ignore_index=True)
data_not_test = pd.concat([data_train, data_val], ignore_index=True)
data_test = pd.concat([data_pos_test, data_neg_test], ignore_index=True)

print(f"Numb train/val/test samples: {len(data_train)} / {len(data_val)} / {len(data_test)}")

print(data_train.shape)

Numb train/val/test samples: 183072 / 58252 / 3816
(183072, 180)


## REMOVE FEATURE CONTAINING ONLY 1 VALUES

## FILTER CORRELATION MATRIX

In [128]:
corr_pd = abs(data_not_test.corr()['TARGET']).sort_values()
corr_pd = corr_pd[1:]

In [129]:
corr_pd.describe()

count    198.000000
mean       0.020818
std        0.073409
min        0.000021
25%        0.002392
50%        0.007945
75%        0.023065
max        1.000000
Name: TARGET, dtype: float64

In [130]:
corr_dict = dict(corr_pd)

In [131]:
thres = 0.002392
drop_ft = []
for k, v in corr_dict.items():
    if v <= thres:
        drop_ft += [k]
len(drop_ft)


50

In [132]:
data_train = data_train.drop(drop_ft, axis=1)
data_val = data_val.drop(drop_ft, axis=1)
data_test = data_test.drop(drop_ft, axis=1)
data_not_test = data_not_test.drop(drop_ft, axis=1)

## SPLIT DATASET

In [252]:
##### REMOVE 1-VALUE FT #####
remove_ft = []
for col in list(data_not_test.drop('TARGET', axis=1).columns):
    if data_not_test[col].sum() == 0 or data_not_test[col].sum() == len(data_not_test[col]):
        remove_ft += [col]

print(remove_ft)        
print(len(remove_ft))

data_train = data_train.drop(remove_ft, axis=1)
data_val = data_val.drop(remove_ft, axis=1)
data_test = data_test.drop(remove_ft, axis=1)
data_not_test = data_not_test.drop(remove_ft, axis=1)

['FLAG_MOBIL', 'FLAG_DOCUMENT_2', 'NAME_INCOME_TYPE_Maternity leave', 'NAME_FAMILY_STATUS_Unknown']
4


In [253]:
X_train = data_train.drop(['TARGET'], axis = 1)
X_test = data_test.drop(['TARGET'], axis = 1)
X_val = data_val.drop(['TARGET'], axis = 1)
y_train = data_train['TARGET']
y_test = data_test['TARGET']
y_val = data_val['TARGET']
X_not_test = data_not_test.drop(['TARGET'], axis = 1)
y_not_test = data_not_test['TARGET']
print(X_train.shape)

(183072, 175)


## TRAIN
Run classifier on both RandomForest and Logistic Regression

In [254]:
rf = RandomForestClassifier(class_weight='balanced')
lg = LogisticRegression(class_weight='balanced')

rf.fit(X_train, y_train)
lg.fit(X_train, y_train)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

## Evaluate

In [154]:
def calculate_metric(gtnp, pdnp, score=None):
    # input are numpy vector
    total_samples = len(gtnp)
    #print(f"Total sample: {total_samples}")
    total_correct = np.sum(gtnp == pdnp)
    accuracy = total_correct / total_samples
    gt_pos = np.where(gtnp == 1)[0]
    gt_neg = np.where(gtnp == 0)[0]
    TP = np.sum(pdnp[gt_pos])
    TN = np.sum(1 - pdnp[gt_neg])
    FP = np.sum(pdnp[gt_neg])
    FN = np.sum(1 - pdnp[gt_pos])
    if score is not None:
        AUC = roc_auc_score(gtnp, score)
        lr_precision, lr_recall, _ = precision_recall_curve(gtnp, score)
        AUC_PR = auc(lr_recall, lr_precision)
    else:
        AUC = roc_auc_score(gtnp, pdnp)
        ACU_PR = -1

    
    
    precision = TP / (TP+FP)
    recall = TP/(TP+FN)
    f1 = 2*precision*recall/(precision+recall)
    metrics = {}
    metrics['accuracy'] = str(accuracy)
    metrics['precision'] = str(precision)
    metrics['recall'] = str(recall)
    metrics['f1'] = str(f1)
    metrics['tp'] = str(int(TP))
    metrics['tn'] = str(int(TN))
    metrics['fp'] = str(int(FP))
    metrics['fn'] = str(int(FN))
    metrics['auc'] = str(AUC)
    metrics['auc_pr'] = str(AUC_PR)

    return metrics

### RF Evaluation

In [255]:
##### TRAIN PREDICTION #####
rf_train_predict = rf.predict(X_train)
rf_train_score = rf.predict_proba(X_train)[:,1] # score of higher label
rf_train_metric = calculate_metric(y_train, rf_train_predict, rf_train_score)

##### VALIDATE PREDICTION #####
rf_val_predict = rf.predict(X_val)
rf_val_score = rf.predict_proba(X_val)[:,1] # score of higher label
rf_val_metric = calculate_metric(y_val, rf_val_predict, rf_val_score)

##### TEST PREDICTION #####
rf_test_predict = rf.predict(X_test)
rf_test_score = rf.predict_proba(X_test)[:,1]
rf_test_metric = calculate_metric(y_test, rf_test_predict, rf_test_score)

### Logistic Regression Evaluation

In [256]:
##### TRAIN PREDICTION #####
lg_train_predict = lg.predict(X_train)
lg_train_score = lg.predict_proba(X_train)[:,1] # score of higher label
lg_train_metric = calculate_metric(y_train, lg_train_predict, lg_train_score)

##### VALIDATE PREDICTION #####
lg_val_predict = lg.predict(X_val)
lg_val_score = lg.predict_proba(X_val)[:,1] # score of higher label
lg_val_metric = calculate_metric(y_val, lg_val_predict, lg_val_score)

##### TEST PREDICTION #####
lg_test_predict = lg.predict(X_test)
lg_test_score = lg.predict_proba(X_test)[:,1]
lg_test_metric = calculate_metric(y_test, lg_test_predict, lg_test_score)

In [257]:
result = {}
result['info'] = 'filtered app, not remove corr, 175 ft, using all numeric, encoded, dropna, classweight balanced, negative:positive = 11:1'
result['RF'] = {}
result['LG'] = {}
result['RF']['train'] = rf_train_metric
result['RF']['val'] = rf_val_metric
result['RF']['test'] = rf_test_metric
result['LG']['train'] = lg_train_metric
result['LG']['val'] = lg_val_metric
result['LG']['test'] = lg_test_metric

In [240]:
result['LG']

{'train': {'accuracy': '0.6340987252088021',
  'precision': '0.1187440034888792',
  'recall': '0.5280428205724924',
  'f1': '0.19388743306368916',
  'tp': '6807',
  'tn': '91283',
  'fp': '50518',
  'fn': '6084',
  'auc': '0.6204540159972532',
  'auc_pr': '0.11833374852784538'},
 'val': {'accuracy': '0.6411093492522464',
  'precision': '0.07174887892376682',
  'recall': '0.5164494103041589',
  'f1': '0.1259937911713485',
  'tp': '832',
  'tn': '19788',
  'fp': '10764',
  'fn': '779',
  'auc': '0.6065156347700551',
  'auc_pr': '0.06822275309882335'},
 'test': {'accuracy': '0.5899503722084367',
  'precision': '0.600974930362117',
  'recall': '0.5353598014888338',
  'f1': '0.5662729658792651',
  'tp': '863',
  'tn': '1039',
  'fp': '573',
  'fn': '749',
  'auc': '0.6218428473791477',
  'auc_pr': '0.5943093298108586'}}

## TEST

In [258]:
filename = 'metric/result_03.json'
with open(filename, 'w') as f:
    json.dump(result, f)

In [205]:
print(encoded_data.shape)

(201103, 127)


In [206]:
encoded_data_test = encoded_data_test.drop(drop_numerical_ft, axis=1)
print(encoded_data_test.shape)

(48744, 124)


In [204]:
data_test = pd.read_csv('data/filtered_merged_application_test.csv')
SK_ID_CURR = data_test['SK_ID_CURR']
data_test = data_test.drop(['SK_ID_CURR'], axis=1) if 'SK_ID_CURR' in list(data_test.columns) else data_test.copy()
data_test = data_test.drop(['DAYS_EMPLOYED_ANOM'], axis=1) if 'DAYS_EMPLOYED_ANOM' in list(data_test.columns) else data_test.copy()
data_test = data_test.drop(['Unnamed: 0'], axis=1) if 'Unnamed: 0' in list(data_test.columns) else data_test.copy()
encoded_data_test = pd.get_dummies(data_test)

In [140]:
drop_ft_test = [x for x in list(encoded_data_test.columns) if x in drop_ft]
encoded_data_test = encoded_data_test.drop(drop_ft_test, axis=1)
print(encoded_data_test.shape)

(48744, 148)


In [208]:
encoded_data_test_fillna = encoded_data_test.fillna(encoded_data_test.mean())

In [214]:
data_test_score = rf.predict_proba(encoded_data_test_fillna)[:,1] # score of higher label

In [215]:
submit = pd.DataFrame({'SK_ID_CURR': SK_ID_CURR, 'TARGET': data_test_score})
submit.to_csv('data/submit_select_numerical_rf.csv', index=False)

In [165]:
submit.head()

,SK_ID_CURR,TARGET
0,100001,0.10
1,100005,0.12
2,100013,0.05
3,100028,0.02
4,100038,0.14


## Feature Importance RF

In [233]:
ft_importance = rf.feature_importances_
name_ft = np.asarray(list(X_train.columns))
index_sort = np.argsort(ft_importance)
name_ft[index_sort]

array(['FLAG_DOCUMENT_12', 'NAME_INCOME_TYPE_Pensioner',
       'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_7',
       'ORGANIZATION_TYPE_Industry: type 8', 'NAME_INCOME_TYPE_Student',
       'FLAG_DOCUMENT_17', 'ORGANIZATION_TYPE_Religion',
       'FLAG_DOCUMENT_21', 'ORGANIZATION_TYPE_Trade: type 5',
       'ORGANIZATION_TYPE_Industry: type 6',
       'ORGANIZATION_TYPE_Industry: type 13', 'FLAG_DOCUMENT_15',
       'NAME_EDUCATION_TYPE_Academic degree', 'FLAG_EMP_PHONE',
       'ORGANIZATION_TYPE_Trade: type 4',
       'ORGANIZATION_TYPE_Industry: type 10', 'FLAG_DOCUMENT_20',
       'FLAG_CONT_MOBILE', 'FLAG_DOCUMENT_19',
       'NAME_TYPE_SUITE_Group of people', 'ORGANIZATION_TYPE_Cleaning',
       'ORGANIZATION_TYPE_Legal Services',
       'ORGANIZATION_TYPE_Transport: type 1', 'ORGANIZATION_TYPE_Culture',
       'OCCUPATION_TYPE_IT staff', 'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14',
       'ORGANIZATION_TYPE_Trade: type 6', 'FLAG_DOCUMENT_11',
       'ORGANIZATION_TYPE_Tra

## FIND PVALUES

In [170]:
import scipy.stats as stat

X = X_train
#### Get p-values for the fitted model ####
denom = (2.0*(1.0+np.cosh(lg.decision_function(X))))
denom = np.tile(denom,(X.shape[1],1)).T
F_ij = np.dot((X/denom).T,X) ## Fisher Information Matrix
Cramer_Rao = np.linalg.inv(F_ij) ## Inverse Information Matrix
sigma_estimates = np.sqrt(np.diagonal(Cramer_Rao))
z_scores = lg.coef_[0]/sigma_estimates # z-score for eaach model coefficient
p_values = [stat.norm.sf(abs(x))*2 for x in z_scores] ### two tailed test for p-values

In [174]:
index_sort = np.argsort(p_values)

In [179]:
pnp = np.asarray(p_values)
index_sort[0:8]

array([ 6,  8,  7, 67, 11, 84, 86, 12])

In [181]:
name_ft = np.asarray(list(X_train.columns))
name_ft[index_sort]

array(['AMT_CREDIT', 'AMT_GOODS_PRICE', 'AMT_ANNUITY',
       'AVG_DIF_AMT_PAYMENT', 'DAYS_EMPLOYED', 'DIF_AMT_CREDIT_mean',
       'DIF_AMT_CREDIT_max', 'DAYS_REGISTRATION', 'DIF_AMT_CREDIT_sum',
       'DAYS_BIRTH', 'DAYS_LAST_PHONE_CHANGE', 'DAYS_ID_PUBLISH',
       'AMT_INCOME_TOTAL', 'DIF_AMT_CREDIT_min',
       'DIF_DAYS_INSTALMENT_sum', 'DIF_DAYS_INSTALMENT_max',
       'DIF_DAYS_INSTALMENT_min', 'DIF_DAYS_INSTALMENT_mean',
       'OWN_CAR_AGE', 'HOUR_APPR_PROCESS_START',
       'AMT_REQ_CREDIT_BUREAU_YEAR', 'AMT_REQ_CREDIT_BUREAU_MON',
       'CODE_GENDER', 'REGION_RATING_CLIENT_W_CITY',
       'REGION_RATING_CLIENT', 'OBS_30_CNT_SOCIAL_CIRCLE',
       'OBS_60_CNT_SOCIAL_CIRCLE', 'FLAG_WORK_PHONE', 'EXT_SOURCE_3',
       'OCCUPATION_TYPE_Laborers', 'EXT_SOURCE_2',
       'DEF_30_CNT_SOCIAL_CIRCLE', 'REG_CITY_NOT_WORK_CITY',
       'FLAG_OWN_CAR', 'DEF_60_CNT_SOCIAL_CIRCLE', 'FLAG_DOCUMENT_3',
       'REG_CITY_NOT_LIVE_CITY', 'LIVE_CITY_NOT_WORK_CITY',
       'FLAG_OWN_REALTY', 

In [182]:
corr_dict

{'NAME_TYPE_SUITE_Group of people': 2.063628898692377e-05,
 'ORGANIZATION_TYPE_Religion': 4.796115032605099e-05,
 'NAME_FAMILY_STATUS_Separated': 5.2539516418956466e-05,
 'ORGANIZATION_TYPE_Industry: type 11': 0.00011261576799478748,
 'ORGANIZATION_TYPE_Business Entity Type 2': 0.00011339888206571532,
 'ORGANIZATION_TYPE_Mobile': 0.000133338115729271,
 'ORGANIZATION_TYPE_Industry: type 13': 0.00014279378774093075,
 'ORGANIZATION_TYPE_Industry: type 8': 0.00020001708938370376,
 'NAME_EDUCATION_TYPE_Incomplete higher': 0.0002026254647917751,
 'AMT_REQ_CREDIT_BUREAU_WEEK': 0.00020450652511080501,
 'FLAG_DOCUMENT_21': 0.00020660373445054636,
 'NAME_TYPE_SUITE_Spouse, partner': 0.00034637732009811087,
 'CNT_FAM_MEMBERS': 0.00038688907949504733,
 'WEEKDAY_APPR_PROCESS_START_WEDNESDAY': 0.0005072280845577627,
 'WEEKDAY_APPR_PROCESS_START_FRIDAY': 0.0005150031094679311,
 'FLAG_DOCUMENT_20': 0.0005599435468385759,
 'ORGANIZATION_TYPE_Industry: type 7': 0.0005885529132414816,
 'ORGANIZATION_TYPE

In [207]:
remove_ft_test = [x for x in list(encoded_data_test.columns) if x in remove_ft]
encoded_data_test = encoded_data_test.drop(remove_ft_test, axis=1)
print(encoded_data_test.shape)

(48744, 122)
